In [5]:
import pandas, numpy

training_data = pandas.DataFrame(numpy.random.rand(3, 4), columns=list('ABCD')).set_index('A')
training_data = training_data.sample(n=training_data.shape[0])
training_data

,B,C,D
A,,,
0.830408,0.194983,0.481360,0.593872
0.162258,0.786237,0.628161,0.489599
0.455702,0.228044,0.749182,0.907533


In [7]:
import torch

t: torch.Tensor = torch.from_numpy(training_data.values)
t

tensor([[ 0.1950,  0.4814,  0.5939],
        [ 0.7862,  0.6282,  0.4896],
        [ 0.2280,  0.7492,  0.9075]], dtype=torch.float64)

In [10]:
t = t.float()

In [ ]:
import torch.nn as nn
import torch.nn.functional as functional
import torch.optim as optim

linear_model = nn.Linear(training_data.shape[1] - 1)
def loss()
loss = torch.sqrt(nn.MSELoss())
optimizer = optim.LBFGS(linear_model.parameters())